## Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2
import yaml

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

## Hyperparameter Setting

In [3]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

## Fixed RandomSeed

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing
    한글 폴더 변경필요

In [44]:
FolderPath = r'C:\Users\wjswp\Dacon\papering_data\train'
os.chdir(FolderPath)
for idx,i in enumerate(os.listdir(FolderPath)):
    os.rename(i,i.replace(i,str(idx)))


In [45]:
all_img_list = glob.glob(FolderPath+'/*/*')
all_img_list[:5]

['C:\\Users\\wjswp\\Dacon\\papering_data\\train\\0\\0.png',
 'C:\\Users\\wjswp\\Dacon\\papering_data\\train\\0\\1.png',
 'C:\\Users\\wjswp\\Dacon\\papering_data\\train\\0\\10.png',
 'C:\\Users\\wjswp\\Dacon\\papering_data\\train\\0\\11.png',
 'C:\\Users\\wjswp\\Dacon\\papering_data\\train\\0\\2.png']

In [46]:
all_img_list[0].split('\\')[-2]

'0'

In [47]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[-2])

In [48]:
df.tail()

,img_path,label
3452,C:\Users\wjswp\Dacon\papering_data\train\9\56.png,9
3453,C:\Users\wjswp\Dacon\papering_data\train\9\6.png,9
3454,C:\Users\wjswp\Dacon\papering_data\train\9\7.png,9
3455,C:\Users\wjswp\Dacon\papering_data\train\9\8.png,9
3456,C:\Users\wjswp\Dacon\papering_data\train\9\9.png,9


In [75]:
dic = {}
for idx,i in enumerate(list(df.label.unique())):
    dic[idx] = i
dic

with open('labelChanger.yaml','w',encoding='utf-8') as f:
    yaml.dump(dic,f,allow_unicode=True)
    f.close()

{0: '가구수정',
 1: '걸레받이수정',
 2: '곰팡이',
 3: '꼬임',
 4: '녹오염',
 5: '들뜸',
 6: '면불량',
 7: '몰딩수정',
 8: '반점',
 9: '석고수정',
 10: '오염',
 11: '오타공',
 12: '울음',
 13: '이음부불량',
 14: '창틀,문틀수정',
 15: '터짐',
 16: '틈새과다',
 17: '피스',
 18: '훼손'}

In [21]:
with open('labelChanger.yaml','r',encoding='utf-8') as f:
    text = yaml.load(f,Loader=yaml.FullLoader)
    # print(text)
text.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18])

In [29]:
# reverse_text = {v:k for k,v in text.items()}
# df['label_index'] = df.apply(lambda x: reverse_text[x['label']],axis=1)
# df.drop(columns='label')

,img_path,label_index
0,C:\Users\wjswp\Dacon\papering_data\train\가구수정\...,0
1,C:\Users\wjswp\Dacon\papering_data\train\가구수정\...,0
2,C:\Users\wjswp\Dacon\papering_data\train\가구수정\...,0
3,C:\Users\wjswp\Dacon\papering_data\train\가구수정\...,0
4,C:\Users\wjswp\Dacon\papering_data\train\가구수정\...,0
...,...,...
3452,C:\Users\wjswp\Dacon\papering_data\train\훼손\99...,18
3453,C:\Users\wjswp\Dacon\papering_data\train\훼손\99...,18
3454,C:\Users\wjswp\Dacon\papering_data\train\훼손\99...,18
3455,C:\Users\wjswp\Dacon\papering_data\train\훼손\99...,18


In [49]:
im = r'C:\Users\wjswp\Dacon\papering_data\train\0\0.png'
ex = cv2.imread(rf'{im}')
ex

array([[[115, 114, 110],
        [116, 115, 111],
        [114, 113, 109],
        ...,
        [128, 128, 134],
        [129, 129, 135],
        [125, 125, 131]],

       [[114, 113, 109],
        [114, 113, 109],
        [117, 116, 112],
        ...,
        [130, 130, 136],
        [130, 130, 136],
        [128, 128, 134]],

       [[116, 115, 111],
        [117, 116, 112],
        [116, 115, 111],
        ...,
        [130, 130, 136],
        [128, 128, 134],
        [127, 127, 133]],

       ...,

       [[110, 108, 107],
        [110, 108, 107],
        [111, 109, 108],
        ...,
        [130, 128, 136],
        [131, 128, 137],
        [134, 131, 140]],

       [[112, 110, 109],
        [110, 108, 107],
        [109, 107, 106],
        ...,
        [129, 128, 136],
        [128, 127, 136],
        [129, 128, 136]],

       [[112, 110, 109],
        [110, 108, 107],
        [110, 108, 107],
        ...,
        [128, 128, 134],
        [129, 129, 135],
        [127, 127, 133]]

In [50]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

## Label-Encoding

In [51]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

## CustomDataset

In [52]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [53]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [55]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

In [56]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

## Train

In [65]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(torch.int32)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [64]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(torch.int32)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

## Run!!

In [66]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 6.00 GiB total capacity; 5.28 GiB already allocated; 0 bytes free; 5.37 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Inference

In [21]:
test = pd.read_csv('./test.csv')

In [22]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [23]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [25]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

## Submission

In [26]:
submit = pd.read_csv('./sample_submission.csv')

In [28]:
submit['label'] = preds

In [29]:
submit.to_csv('./baseline_submit.csv', index=False)